In [4]:
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
import requests
import logging

In [5]:
lib_df = pd.read_csv("../data/mergent_and_library/ffxlib_all.csv")
lib_df.head()


,Business Name,Address,City,State,ZIP,Name,Title,Employee Size,Home Based Business,Franchise Type
0,Centralcare Nursing Inc,7010 Little River Tpke Ste 335,Annandale,VA,22003.0,Howard Hankerson,Executive Officer,20.0,No,NaN
1,Genesis HealthCare,2030 Westmoreland St,Falls Church,VA,22043.0,,NaN,19.0,No,Chain Location
2,Immediate Labs LLC,3704 Keefer Ct,Fairfax,VA,22033.0,Jin Feng,Owner,8.0,No,NaN
3,Keener Insurance Inc,13880 Braddock Rd Ste 305,Centreville,VA,20121.0,Elsi Balbuena,Executive Officer,5.0,No,NaN
4,American Insurances,4221 Walney Rd Ste 202,Chantilly,VA,20151.0,Sonya Keppert,Owner,5.0,No,NaN


In [ ]:
lib_df['full_address'] = lib_df['Address'] + ', ' + lib_df['City'] + ", " + lib_df['State'] + ' ' + lib_df['ZIP'].astype(str).str.split('.').str[0]

In [6]:
lib_df['ZIP'] = lib_df['ZIP'].fillna(0)
lib_df = lib_df.rename(columns={"Address": "Street address"})

In [ ]:

lib_df.head()

In [ ]:
lib_df['ZIP'].unique()

In [7]:
lib_df['ZIP'] = lib_df['ZIP'].astype(float).astype(int)
lib_df.head()

,Business Name,Street address,City,State,ZIP,Name,Title,Employee Size,Home Based Business,Franchise Type
0,Centralcare Nursing Inc,7010 Little River Tpke Ste 335,Annandale,VA,22003,Howard Hankerson,Executive Officer,20.0,No,NaN
1,Genesis HealthCare,2030 Westmoreland St,Falls Church,VA,22043,,NaN,19.0,No,Chain Location
2,Immediate Labs LLC,3704 Keefer Ct,Fairfax,VA,22033,Jin Feng,Owner,8.0,No,NaN
3,Keener Insurance Inc,13880 Braddock Rd Ste 305,Centreville,VA,20121,Elsi Balbuena,Executive Officer,5.0,No,NaN
4,American Insurances,4221 Walney Rd Ste 202,Chantilly,VA,20151,Sonya Keppert,Owner,5.0,No,NaN


In [ ]:
lib_df_addr = pd.DataFrame(lib_df[['Street address', "City", "State", "ZIP"]])
lib_df_addr.head()

In [8]:
import numpy as np
dfs = np.array_split(lib_df, (len(lib_df) / 10000) + 1)
#dfs[0]

In [9]:
dfs[0]

,Business Name,Street address,City,State,ZIP,Name,Title,Employee Size,Home Based Business,Franchise Type
0,Centralcare Nursing Inc,7010 Little River Tpke Ste 335,Annandale,VA,22003,Howard Hankerson,Executive Officer,20.0,No,NaN
1,Genesis HealthCare,2030 Westmoreland St,Falls Church,VA,22043,,NaN,19.0,No,Chain Location
2,Immediate Labs LLC,3704 Keefer Ct,Fairfax,VA,22033,Jin Feng,Owner,8.0,No,NaN
3,Keener Insurance Inc,13880 Braddock Rd Ste 305,Centreville,VA,20121,Elsi Balbuena,Executive Officer,5.0,No,NaN
4,American Insurances,4221 Walney Rd Ste 202,Chantilly,VA,20151,Sonya Keppert,Owner,5.0,No,NaN
...,...,...,...,...,...,...,...,...,...,...
8397,TCMP Staffing Service 2 LLC,6551 Loisdale Ct Ste 500,Springfield,VA,22150,Michael Perry,Executive Officer,17.0,No,NaN
8398,Nvepc,3337 Duke St,Burke,VA,22009,,NaN,14.0,No,NaN
8399,The Leland House - UMFS,13525 Leland Rd,Centreville,VA,20120,,NaN,18.0,No,NaN
8400,Youth For Tomorrow,6800 Backlick Rd Ste 300,Springfield,VA,22150,,NaN,18.0,No,NaN


In [8]:
chunk_names = []
l = len(dfs)
output_dir = "geocoded"
for i in range(l):
        save_name = os.path.join(
            output_dir,
            "ffx_geocoded_{i}.csv".format(
                i=str(i).zfill(int(len(str(len(dfs)))))
            ),  
        )
        try:
            dfs[i].to_csv(save_name)
        except:
            pass
        logging.debug(os.path.isfile(save_name))
        if os.path.isfile(save_name):
            chunk_names.append(save_name)

In [ ]:
pbar = tqdm(chunk_names)
print(pbar)
for chunk in pbar:
    save_name = "{chunk_name}_geocoded.csv".format(
        chunk_name=chunk.split(".csv")[0]
    )
    logging.debug(save_name)
    pbar.set_description(save_name)

    files = {
        "addressFile": open(chunk, "r"),
        "benchmark": (None, "2020"),
    }

    response = requests.post(
        "https://geocoding.geo.census.gov/geocoder/locations/addressbatch",
        files=files,
    )
    logging.debug("Saving to: %s" % save_name)
    with open(save_name, "wb") as f:
        f.write(response.content)

    logging.debug("Fixing malformed headers")
    with open(save_name, "r", encoding="utf-8") as file:
        data = file.readlines()

    data[0] = "index,input,match,non_exact,street,coordinate,tiger,lr\n"

    with open(save_name, "w", encoding="utf-8") as file:
        file.writelines(data)

logging.info("Deleting all non_geocoded files")

for file in os.listdir(output_dir):
    if "_geocoded" not in file:
        os.remove(os.path.join(output_dir, file))



geocoded/ffx_geocoded_0_geocoded.csv:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]


In [36]:
import os


PATH_1 = "geocoded"
PATH_2 = "geocoded/_orig"

ls_1 = sorted(os.listdir(PATH_1))[2:]
ls_2 = sorted(os.listdir(PATH_2))


#print(ls_1)

for i in range(len(ls_1)):
    df1 = pd.read_csv(os.path.join(PATH_1, f"ffx_geocoded_{i}_geocoded.csv"))
    df2 = pd.read_csv(os.path.join(PATH_2, f"ffx_geocoded_{i}.csv"))
    merged_df = df2.merge(df1, left_on="Unnamed: 0", right_on='index')[['Business Name', 'Street address', 'City', 'State', 'ZIP', 'Name', 'Title', 'Employee Size', 'Home Based Business', 'Franchise Type', 'coordinate']]
    merged_df.to_csv(f"geocoded_complete/ffx_geocoded_{i}.csv")

    
    
    


